<a href="https://colab.research.google.com/github/ConradBitt/processamento_linguagem_natural/blob/master/Criando_Classificador_com_Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Este notebook é uma complementação do [Word2vec_Treinando_word_embedding](https://github.com/ConradBitt/processamento_linguagem_natural/blob/master/Word2vec_Treinando_word_embedding.ipynb) onde treinamos modelos. Aqui vamos utiliza-los para criar um classificador.


# Importações de módulos e instalando idioma do spacy

In [1]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib as mpl
import spacy


from matplotlib import pyplot as plt

sns.set_context('talk')

# Executar apenas uma vez e depois reiniciar o ambiente de execução.
!python -m spacy download pt_core_news_sm

print('\n\n~ Versão python ~')
!python --version

print('\n\n~ Versões dos Módulos ~')
print(f'Pandas: {pd.__version__}')
print(f'Seaborn: {sns.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Matplotlib: {mpl.__version__}')
print(f'Spacy: {spacy.__version__}')

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


~ Versão python ~
Python 3.7.10


~ Versões dos Módulos ~
Pandas: 1.1.5
Seaborn: 0.11.1
Numpy: 1.19.5
Matplotlib: 3.2.2
Spacy: 2.2.4


## Carregando idioma

In [2]:
# carregando o idioma
nlp = spacy.load('pt_core_news_sm', disable=['parser','ner','tagger','textcat'])


## Montando Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carregando modelos já treinados com gensim

In [4]:
from gensim.models import KeyedVectors

In [5]:
cbow = '/content/drive/MyDrive/Colab Notebooks/word2vec/modelos/w2v_cbow_300.txt'
skipgram = '/content/drive/MyDrive/Colab Notebooks/word2vec/modelos/w2v_skipgram_300.txt'

w2v_cbow = KeyedVectors.load_word2vec_format(cbow)
w2v_skipgram = KeyedVectors.load_word2vec_format(cbow)

# Pre processamento dos dados de entrada

## Importando dados de treino e teste

In [6]:
treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/word2vec/dados/treino.csv')
teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/word2vec/dados/teste.csv')

---
## Tokenizador
``tokenizador`` é uma função que recebe uma string qualquer e a partir dela cria um objeto do tipo ``Doc`` do SpaCy. Esta estrutura facilita análise de alguns textos, por exemplo, se é stopword ou não, se é alphanumerico ou não, etc. Por fim vamos adicionar o texto à uma lista de tokens validados pelo pre processamento e retorna-la.

In [7]:

def tokenizador(texto):
  """
  Esta função recebe um texto
  cria um doc do spacy através do texto recebido
  remove stopwords, retorna alphanumericos, deixa minusculo
  tokeniza o texto e retorna uma lista de tokens
  """
  tokens_validos = []
  doc = nlp(texto)
  for token in doc: 
    # variável de validação
    # garante que o token não é stopword e é alphanumerico
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())
      
  return tokens_validos

texto_de_teste = 'Rio de Janeiro 908175!@&$*!@éé uma cidade maravilhosa!'

#testando tokenizador
tokens_teste = tokenizador(texto_de_teste)
print(tokens_teste)

['rio', 'janeiro', 'cidade', 'maravilhosa']


note que foi retirado números, caracteres não alphabeticos e stopwords.

---
## Combinação linear de tokens

A ``combinacao_linear_tokens`` incorpora os tokens à um espaço vetorial. Basicamente se tem um espaço vetorial de ordem bem grande, 300, 500, 1000, ou mais. Cada token pode ser representado neste espaço vetorial e portanto o modelo cria uma combinação linear (soma) de vetores para expressar a frase. 

Como existe mais de uma arquitetura de modelo (CBOW e SG) será adicionado um parâmetro para que o modelo seja fornecido para produzir a combinação linear.

In [8]:

def combinacao_linear_tokens(tokens, modelo):
  token_vetorizado = np.zeros(300)

  for token in tokens:
    try:
      # soma os vetores de cada token pra construir um vetor da frase.
      token_vetorizado += modelo.get_vector(token)
    except KeyError:
      #print(f'\033[31m KeyError: O token "{token}" não esta no vocabulario. \033[0;0m')
      pass

  return token_vetorizado


testando combinação linear:

In [9]:
token_teste_cbow = combinacao_linear_tokens(tokens_teste, w2v_cbow)
token_teste_skipgram = combinacao_linear_tokens(tokens_teste, w2v_skipgram)

print(f'CBOW {token_teste_cbow.shape}\nSkipGram {token_teste_skipgram.shape}')

CBOW (300,)
SkipGram (300,)


conseguimos transformar em vetores a frase utilizando dois modelos: Skip Gram e Continous Bag of Words.

---

## Tensorizando frases

Essa função vai ser responsável por pegar todas as frases e vetoriza-las, logo teremos uma matriz de 300 colunas e linhas iguais à quantidade de observações disponííveis, isto é, de titulos de notícias.

In [10]:
def tensoriza(array_texto, modelo):
  """
  Variável array_texto é uma lista de palavras
  onde cada palavra será tokenizada e em seguida
  criada a sua representação vetorial e por fim,
  armazenada numa matriz. Matriz que é retornada.
  """
  x = len(array_texto) # quantidade de observações, titulos
  y = 300  # colunas, ordem do espaço vetorial

  matriz = np.zeros((x,y))

  for indice in range(x):
    palavras = tokenizador(array_texto.iloc[indice])
    matriz[indice] = combinacao_linear_tokens(palavras, modelo)
    
  return matriz 

# Avaliando modelos



## Vetorizando treino e teste com modelo CBOW

Este processo pode demorar um pouco, pois o processo será realizado 90000 vezes.

In [11]:
treino.title.shape

(90000,)

In [12]:
%%time

tensor_treino_cbow = tensoriza(treino.title, w2v_cbow)
tensor_teste_cbow = tensoriza(teste.title, w2v_cbow)

CPU times: user 1h 45min 17s, sys: 1min 33s, total: 1h 46min 51s
Wall time: 1h 46min 42s


In [13]:
print(tensor_treino_cbow.shape)
print(tensor_teste_cbow.shape)

(90000, 300)
(20513, 300)


### Instenciando modelos do Sci-kit Learn

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

Como temos dois tipos de arquitetura de word2vec, vou criar uma função que irá treinar e ajustar o modelo:

In [24]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  logistic_regression = LogisticRegression(max_iter = 350)
  logistic_regression.fit(x_treino, y_treino)

  categoria_predita = logistic_regression.predict(x_teste)
  print(classification_report(y_teste, categoria_predita))

  return logistic_regression

Usando a função

In [25]:
%%time

lregression_cbow = classificador(w2v_cbow,
                                 x_treino = tensor_treino_cbow,
                                 y_treino = treino.category,
                                 x_teste = tensor_teste_cbow,
                                 y_teste = teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513

CPU times: user 1min 13s, sys: 14.4 s, total: 1min 27s
Wall time: 45.4 s


#### Comentario sobre desempenho do CBOW 

> Este é o resultado do modelo, uma revocação média de 80% e uma acurácia média de 79% em prever os títulos. Entretanto, podemos verificar que a precisão na categoria ``ilustrada`` é de 13%, entretanto isso pode ser explicado pela pequena quantidade de amostras, apenas 131. **Por isso neste caso podemos avaliar a média ponderada (``weighted average``) que tem uma precisão ou valor preditivo positivo de 80% e uma revocação ou sensibilidade de 79%**

## Vetorizando treino e teste com modelo Skip Gram

In [29]:
%%time

tensor_treino_skipgram = tensoriza(treino.title, w2v_skipgram)
tensor_teste_skipgram = tensoriza(teste.title, w2v_skipgram)

CPU times: user 1h 54min 32s, sys: 1min 28s, total: 1h 56min
Wall time: 1h 55min 53s


foi significativamente maior que o CBOW.

In [30]:
print(tensor_treino_skipgram.shape)
print(tensor_teste_skipgram.shape)

(90000, 300)
(20513, 300)


### Usando classificador

In [31]:
%%time

lregression_skipgram = classificador(w2v_skipgram,
                                     x_treino = tensor_treino_skipgram,
                                     y_treino = treino.category,
                                     x_teste = tensor_teste_skipgram,
                                     y_teste = teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513

CPU times: user 1min 13s, sys: 14.1 s, total: 1min 27s
Wall time: 45.4 s


#### Comentando sobre desempenho do skipgram

> Dado que o desempenho das duas arquiteturas de Word2Vec foram similares, neste caso de classificação de títulos o SkipGram não é recomendado, pois o tempo de treino desses 90mil dados foi de 1h 56min contra a 1h 46min do CBOW.

> Apesar do modelo SkipGram ter um desempenho melhor, um preditivo positivo da media ponderada de 82% e uma sensibilidade de 79%, isso vai depender muito do tipo de dado com que se esta trabalhando. 

> Na conclusão do notebook [Word2Vec: Interpretação da linguagem com word embedding](https://github.com/ConradBitt/processamento_linguagem_natural/blob/master/Word2Vec_Interpreta%C3%A7%C3%A3o_da_linguagem_com_word_embedding_.ipynb) discutimos sobre o tamanho dos dados com quais as arquiteturas lidam, isto é, testamos as arquiteturas prevendo titulos com frases até 10 palavras e com artigos inteiros. Verifiquei que: **Dada a natureza do SkipGram prever o contexto a traves de uma palavra quando a palavra pertence à um artigo inteiro é relativamente dificil pra esta arquitetura entregar bons resultados. Ja a caracteristica do CBOW de prever uma palavra dado um contexto de uma frase tem um desempenho muito melhor com dados muitos longos.**
